##Домашнее задание №1
###На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [1]:
import numpy as np
import pandas as pd

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43, 156, 3345, 10, 15, 1134]
bought_list = [156, 611, 10, 24, 521]
prices_recommended = [256, 634, 53, 324, 343, 123, 514, 512, 874, 571, 43, 55, 243, 634, 511]
prices_bought = []

data = pd.DataFrame({"user_id": ["u1", "u2", "u3"],
 "recommended_list": [[143, 156, 1134, 991, 27],
                     [1543, 3345, 533, 11, 43],
                     [156, 3345, 10, 15, 1134]],

"bought_list": [[156],
                [611],
                [10, 15, 521]],

"prices_recommended": [[256, 634, 53, 324, 343],
                      [123, 514, 512, 874, 571],
                      [43, 55, 243, 634, 511]],
 "prices_bought": [[632],
                   [115],
                   [240, 640, 387]]});
data

,user_id,recommended_list,bought_list,prices_recommended,prices_bought
0,u1,"[143, 156, 1134, 991, 27]",[156],"[256, 634, 53, 324, 343]",[632]
1,u2,"[1543, 3345, 533, 11, 43]",[611],"[123, 514, 512, 874, 571]",[115]
2,u3,"[156, 3345, 10, 15, 1134]","[10, 15, 521]","[43, 55, 243, 634, 511]","[240, 640, 387]"


In [3]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

     #TODO: Ваш код здесь
    flags = np.isin(recommended_list[:k],bought_list)
    recall = flags.sum()/len(bought_list)

    return recall #Добавьте сюда результат расчета метрики


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)

    #TODO: Ваш код здесь

    flags = np.isin(recommended_list[:k], bought_list)
    precision = (flags * prices_recommended[:k]).sum() / prices_recommended[:k].sum()

    return precision #Добавьте сюда результат расчета метрики


In [4]:
data.apply(lambda x: recall_at_k(x[1], x[2], 5), 1).mean()

0.5555555555555555

In [5]:
data.apply(lambda x: money_recall_at_k(x[1], x[2], x[3], x[4], 5), 1).mean()


0.3279879287428003

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [6]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    #TODO: Ваш код здесь
    flags = np.isin(recommended_list[:k], bought_list)
    if flags.sum() != 0:
      rank = np.where(flags==True)[0][0]+1
      return 1/rank  #Добавьте сюда результат расчета метрики
    else:
      None

In [7]:
data.apply(lambda x: mrr_at_k(x[1], x[2], 5), 1).mean()

0.41666666666666663

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [8]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    #TODO: Ваш код здесь
    dcg_at_k = 0
    idcg_at_k = 0
    flags = np.isin(recommended_list[:k], bought_list)
    if flags.sum() != 0:
      for i in range(len(flags)):
        idcg_at_k += 1/np.log2(i+2)
        if flags[i]:
          dcg_at_k += 1/np.log2(i+2)

      return dcg_at_k/idcg_at_k  #Добавьте сюда результат расчета метрики
    else:
      return 0

In [9]:
data.apply(lambda x: ndcg_at_k(x[1], x[2], 5), 1).mean()

0.17654490573801404